# Fine-tuning Whisper on Speech Pathology Dataset

## Goal

The goal of the Cleft Palate project at Vanderbilt DSI is to classify audio clips of patients' voices as containing hypernasality (a speech impediment) or not. The patients with hypernasality can then be recommended for speech pathology intervention. This is currently evaluated by human speech pathologists, which requires access to these medical providers. Our hope is to train a model that can classify this speech impediment for expedited patient access to a speech pathologist.

Tutorial created with guidance from ["Fine Tuning OpenAI Whisper Model for Audio Classifcation in PyTorch"](https://www.daniweb.com/programming/computer-science/tutorials/540802/fine-tuning-openai-whisper-model-for-audio-classification-in-pytorch)

## Model

We plan to use the Whisper embedings from OpenAI and train a classification model, either using Whisper with a sequence classification head or another classification LLM.

## Data

The data in this notebook is publicly available voice recordings featuring hypernasality and control groups. In the future we hope to train our model on private patient data from Vanderbilt University Medical Center (VUMC).

### Split Data

We need to split our data into train and test sets, then save those for further experiments.

In [2]:
!pip install torch
!pip install datasets
!pip install librosa
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# import libraries
import datasets
from datasets import load_dataset, DatasetDict,  Audio
import pandas as pd
import os
import glob
import librosa
import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from transformers import WhisperModel, WhisperFeatureExtractor, AdamW
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import f1_score, classification_report, accuracy_score

# ==================================================================================================================
# Tiny

In [4]:
data_path = "/workspace/cleft_palate_choja/WAV_PUBLIC_SAMPLES/NOISE"

train_catalog = "/workspace/cleft_palate_choja/train_noise.csv"
test_catalog = "/workspace/cleft_palate_choja/test_noise.csv"
model_checkpoint = "openai/whisper-tiny"

train_metadata = pd.read_csv(train_catalog)
train_df, val_df = train_test_split(train_metadata, test_size = 0.3, random_state = 42)
train_files = train_df["WAV_filename"].tolist()
train_folder = train_df["WAV_folder"].tolist()
train_full_paths = [os.path.join(data_path,train_folder[i], train_files[i]) for i in range(0,len(train_files))]
train_labels = train_df["hypernasality"].tolist()
# val set
val_files = val_df["WAV_filename"].tolist()

val_folder = val_df["WAV_folder"].tolist()

val_full_paths = [os.path.join(data_path,val_folder[i], val_files[i]) for i in range(0,len(val_files))]

val_labels = val_df["hypernasality"].tolist()

test_metadata = pd.read_csv(test_catalog)
# add cols for wav data

# Replace ".mp3" with ".wav" in the "Filename" column
test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
test_metadata['WAV_folder'] = test_metadata['folder'] + "_WAV"

test_files = test_metadata["WAV_filename"].tolist()

test_folder = test_metadata["WAV_folder"].tolist()

test_full_paths = [os.path.join(data_path,test_folder[i], test_files[i]) for i in range(0,len(test_files))]

#test_full_paths

test_labels = test_metadata["hypernasality"].tolist()

train_audio_dataset = datasets.Dataset.from_dict({"audio": train_full_paths,
                                                  "labels":train_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
test_audio_dataset = datasets.Dataset.from_dict({"audio": test_full_paths,
                                                  "labels": test_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
val_audio_dataset = datasets.Dataset.from_dict({"audio": val_full_paths,
                                                 "labels": val_labels }
                                             ).cast_column("audio", Audio(sampling_rate=16_000))
#model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

      inputs = self.text_processor(self.audio_data[index]["audio"]["array"],
                                   return_tensors="pt",
                                   sampling_rate=self.audio_data[index]["audio"]["sampling_rate"])
      input_features = inputs.input_features
      decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

      labels = np.array(self.audio_data[index]['labels'])

      return input_features, decoder_input_ids, torch.tensor(labels)
train_dataset = SpeechClassificationDataset(train_audio_dataset,  feature_extractor)
test_dataset = SpeechClassificationDataset(test_audio_dataset,  feature_extractor)
val_dataset = SpeechClassificationDataset(val_audio_dataset,  feature_extractor)

batch_size = 5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SpeechClassifier(nn.Module):
    def __init__(self, num_labels, encoder):
        super(SpeechClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_features, decoder_input_ids):
        outputs = self.encoder(input_features, decoder_input_ids=decoder_input_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits
num_labels = 2

model = SpeechClassifier(num_labels, encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()
# Define the training function NO VAL
def train(model, train_loader, optimizer, criterion, device, num_epochs):

  for epoch in range(num_epochs):

    model.train()

    for i, batch in enumerate(train_loader):

          input_features, decoder_input_ids, labels = batch

          input_features = input_features.squeeze()
          input_features = input_features.to(device)

          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)

          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)

          optimizer.zero_grad()

          logits = model(input_features, decoder_input_ids)

          loss = criterion(logits, labels)
          loss.backward()

          optimizer.step()

          if (i+1) % 8 == 0:
              print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), 'best_model.pt')
# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        for i, batch in enumerate(train_loader):
            input_features, decoder_input_ids, labels = batch
            input_features = input_features.squeeze()
            input_features = input_features.to(device)
            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)
            labels = labels.view(-1)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            optimizer.zero_grad()
            logits = model(input_features, decoder_input_ids)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0
        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')
        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, Best Accuracy: {best_accuracy:.4f}')
        print("========================================================================================")
def evaluate(model, data_loader,  device):
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
          input_features, decoder_input_ids, labels = batch
          input_features = input_features.squeeze()
          input_features = input_features.to(device)
          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)
          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)
          optimizer.zero_grad()
          logits = model(input_features, decoder_input_ids)
          loss = criterion(logits, labels)
          total_loss += loss.item()
          _, preds = torch.max(logits, 1)
          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds

import librosa
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

/tmp/ipykernel_1499/2197773112.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Batch 8/41, Train Loss: 1.1287
Epoch 1/5, Batch 16/41, Train Loss: 0.8537
Epoch 1/5, Batch 24/41, Train Loss: 0.7160
Epoch 1/5, Batch 32/41, Train Loss: 0.5143
Epoch 1/5, Batch 40/41, Train Loss: 0.3436
Epoch 1/5, Val Loss: 0.2688, Val Accuracy: 0.8652, Val F1: 0.8650, Best Accuracy: 0.8652
Epoch 2/5, Batch 8/41, Train Loss: 0.4391
Epoch 2/5, Batch 16/41, Train Loss: 0.0375
Epoch 2/5, Batch 24/41, Train Loss: 0.0240
Epoch 2/5, Batch 32/41, Train Loss: 0.0628
Epoch 2/5, Batch 40/41, Train Loss: 0.0135
Epoch 2/5, Val Loss: 0.0816, Val Accuracy: 0.9663, Val F1: 0.9662, Best Accuracy: 0.9663
Epoch 3/5, Batch 8/41, Train Loss: 0.0014
Epoch 3/5, Batch 16/41, Train Loss: 0.0028
Epoch 3/5, Batch 24/41, Train Loss: 0.0171
Epoch 3/5, Batch 32/41, Train Loss: 0.0019
Epoch 3/5, Batch 40/41, Train Loss: 0.0028
Epoch 3/5, Val Loss: 0.0420, Val Accuracy: 0.9888, Val F1: 0.9887, Best Accuracy: 0.9888
Epoch 4/5, Batch 8/41, Train Loss: 0.0010
Epoch 4/5, Batch 16/41, Train Loss: 0.0048
Epoch 

### Validation

Before running the model on the test set, let's examine the validation set and see how our model is doing.

In [5]:
#VALIDATION
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, val_loader, device)

#VALIDATION
print("================")
print("Validtation\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

# TESTING ONLY
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, test_loader, device)

print("================")
print("Testing\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

Validtation


              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.98      1.00      0.99        48

    accuracy                           0.99        89
   macro avg       0.99      0.99      0.99        89
weighted avg       0.99      0.99      0.99        89

0.9887640449438202
Testing


              precision    recall  f1-score   support

           0       0.94      0.89      0.91        36
           1       0.90      0.95      0.92        38

    accuracy                           0.92        74
   macro avg       0.92      0.92      0.92        74
weighted avg       0.92      0.92      0.92        74

0.918918918918919


# ==================================================================================================================
# Small

In [6]:
data_path = "/workspace/cleft_palate_choja/WAV_PUBLIC_SAMPLES/NOISE"

train_catalog = "/workspace/cleft_palate_choja/train_noise.csv"
test_catalog = "/workspace/cleft_palate_choja/test_noise.csv"
model_checkpoint = "openai/whisper-small"

train_metadata = pd.read_csv(train_catalog)
train_df, val_df = train_test_split(train_metadata, test_size = 0.3, random_state = 42)
train_files = train_df["WAV_filename"].tolist()
train_folder = train_df["WAV_folder"].tolist()
train_full_paths = [os.path.join(data_path,train_folder[i], train_files[i]) for i in range(0,len(train_files))]
train_labels = train_df["hypernasality"].tolist()
# val set
val_files = val_df["WAV_filename"].tolist()

val_folder = val_df["WAV_folder"].tolist()

val_full_paths = [os.path.join(data_path,val_folder[i], val_files[i]) for i in range(0,len(val_files))]

val_labels = val_df["hypernasality"].tolist()

test_metadata = pd.read_csv(test_catalog)
# add cols for wav data

# Replace ".mp3" with ".wav" in the "Filename" column
test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
test_metadata['WAV_folder'] = test_metadata['folder'] + "_WAV"

test_files = test_metadata["WAV_filename"].tolist()

test_folder = test_metadata["WAV_folder"].tolist()

test_full_paths = [os.path.join(data_path,test_folder[i], test_files[i]) for i in range(0,len(test_files))]

#test_full_paths

test_labels = test_metadata["hypernasality"].tolist()

train_audio_dataset = datasets.Dataset.from_dict({"audio": train_full_paths,
                                                  "labels":train_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
test_audio_dataset = datasets.Dataset.from_dict({"audio": test_full_paths,
                                                  "labels": test_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
val_audio_dataset = datasets.Dataset.from_dict({"audio": val_full_paths,
                                                 "labels": val_labels }
                                             ).cast_column("audio", Audio(sampling_rate=16_000))
#model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

      inputs = self.text_processor(self.audio_data[index]["audio"]["array"],
                                   return_tensors="pt",
                                   sampling_rate=self.audio_data[index]["audio"]["sampling_rate"])
      input_features = inputs.input_features
      decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

      labels = np.array(self.audio_data[index]['labels'])

      return input_features, decoder_input_ids, torch.tensor(labels)
train_dataset = SpeechClassificationDataset(train_audio_dataset,  feature_extractor)
test_dataset = SpeechClassificationDataset(test_audio_dataset,  feature_extractor)
val_dataset = SpeechClassificationDataset(val_audio_dataset,  feature_extractor)

batch_size = 5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SpeechClassifier(nn.Module):
    def __init__(self, num_labels, encoder):
        super(SpeechClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_features, decoder_input_ids):
        outputs = self.encoder(input_features, decoder_input_ids=decoder_input_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits
num_labels = 2

model = SpeechClassifier(num_labels, encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()
# Define the training function NO VAL
def train(model, train_loader, optimizer, criterion, device, num_epochs):

  for epoch in range(num_epochs):

    model.train()

    for i, batch in enumerate(train_loader):

          input_features, decoder_input_ids, labels = batch

          input_features = input_features.squeeze()
          input_features = input_features.to(device)

          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)

          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)

          optimizer.zero_grad()

          logits = model(input_features, decoder_input_ids)

          loss = criterion(logits, labels)
          loss.backward()

          optimizer.step()

          if (i+1) % 8 == 0:
              print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), 'best_model.pt')
# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        for i, batch in enumerate(train_loader):
            input_features, decoder_input_ids, labels = batch
            input_features = input_features.squeeze()
            input_features = input_features.to(device)
            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)
            labels = labels.view(-1)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            optimizer.zero_grad()
            logits = model(input_features, decoder_input_ids)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0
        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')
        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, Best Accuracy: {best_accuracy:.4f}')
        print("========================================================================================")
def evaluate(model, data_loader,  device):
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
          input_features, decoder_input_ids, labels = batch
          input_features = input_features.squeeze()
          input_features = input_features.to(device)
          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)
          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)
          optimizer.zero_grad()
          logits = model(input_features, decoder_input_ids)
          loss = criterion(logits, labels)
          total_loss += loss.item()
          _, preds = torch.max(logits, 1)
          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds

import librosa
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

/tmp/ipykernel_1499/3089817272.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Batch 8/41, Train Loss: 0.6626
Epoch 1/5, Batch 16/41, Train Loss: 0.3709
Epoch 1/5, Batch 24/41, Train Loss: 0.0620
Epoch 1/5, Batch 32/41, Train Loss: 0.0348
Epoch 1/5, Batch 40/41, Train Loss: 0.3284
Epoch 1/5, Val Loss: 0.1450, Val Accuracy: 0.9326, Val F1: 0.9311, Best Accuracy: 0.9326
Epoch 2/5, Batch 8/41, Train Loss: 0.0330
Epoch 2/5, Batch 16/41, Train Loss: 0.0093
Epoch 2/5, Batch 24/41, Train Loss: 0.0054
Epoch 2/5, Batch 32/41, Train Loss: 0.0507
Epoch 2/5, Batch 40/41, Train Loss: 0.7692
Epoch 2/5, Val Loss: 0.2200, Val Accuracy: 0.9438, Val F1: 0.9428, Best Accuracy: 0.9438
Epoch 3/5, Batch 8/41, Train Loss: 0.0127
Epoch 3/5, Batch 16/41, Train Loss: 0.0080
Epoch 3/5, Batch 24/41, Train Loss: 0.0030
Epoch 3/5, Batch 32/41, Train Loss: 0.0014
Epoch 3/5, Batch 40/41, Train Loss: 0.0555
Epoch 3/5, Val Loss: 0.0309, Val Accuracy: 0.9888, Val F1: 0.9887, Best Accuracy: 0.9888
Epoch 4/5, Batch 8/41, Train Loss: 0.0023
Epoch 4/5, Batch 16/41, Train Loss: 0.0012
Epoch 

In [7]:
#VALIDATION
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, val_loader, device)

#VALIDATION
print("================")
print("Validatation\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

# TESTING ONLY
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, test_loader, device)

print("================")
print("Testing\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

Validtation


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        48

    accuracy                           1.00        89
   macro avg       1.00      1.00      1.00        89
weighted avg       1.00      1.00      1.00        89

1.0
Testing


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        38

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74

1.0


# ==================================================================================================================
# Medium

In [9]:
data_path = "/workspace/cleft_palate_choja/WAV_PUBLIC_SAMPLES/NOISE"

train_catalog = "/workspace/cleft_palate_choja/train_noise.csv"
test_catalog = "/workspace/cleft_palate_choja/test_noise.csv"
model_checkpoint = "openai/whisper-medium"

train_metadata = pd.read_csv(train_catalog)
train_df, val_df = train_test_split(train_metadata, test_size = 0.3, random_state = 42)
train_files = train_df["WAV_filename"].tolist()
train_folder = train_df["WAV_folder"].tolist()
train_full_paths = [os.path.join(data_path,train_folder[i], train_files[i]) for i in range(0,len(train_files))]
train_labels = train_df["hypernasality"].tolist()
# val set
val_files = val_df["WAV_filename"].tolist()

val_folder = val_df["WAV_folder"].tolist()

val_full_paths = [os.path.join(data_path,val_folder[i], val_files[i]) for i in range(0,len(val_files))]

val_labels = val_df["hypernasality"].tolist()

test_metadata = pd.read_csv(test_catalog)
# add cols for wav data

# Replace ".mp3" with ".wav" in the "Filename" column
test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
test_metadata['WAV_folder'] = test_metadata['folder'] + "_WAV"

test_files = test_metadata["WAV_filename"].tolist()

test_folder = test_metadata["WAV_folder"].tolist()

test_full_paths = [os.path.join(data_path,test_folder[i], test_files[i]) for i in range(0,len(test_files))]

#test_full_paths

test_labels = test_metadata["hypernasality"].tolist()

train_audio_dataset = datasets.Dataset.from_dict({"audio": train_full_paths,
                                                  "labels":train_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
test_audio_dataset = datasets.Dataset.from_dict({"audio": test_full_paths,
                                                  "labels": test_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
val_audio_dataset = datasets.Dataset.from_dict({"audio": val_full_paths,
                                                 "labels": val_labels }
                                             ).cast_column("audio", Audio(sampling_rate=16_000))
#model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

      inputs = self.text_processor(self.audio_data[index]["audio"]["array"],
                                   return_tensors="pt",
                                   sampling_rate=self.audio_data[index]["audio"]["sampling_rate"])
      input_features = inputs.input_features
      decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

      labels = np.array(self.audio_data[index]['labels'])

      return input_features, decoder_input_ids, torch.tensor(labels)
train_dataset = SpeechClassificationDataset(train_audio_dataset,  feature_extractor)
test_dataset = SpeechClassificationDataset(test_audio_dataset,  feature_extractor)
val_dataset = SpeechClassificationDataset(val_audio_dataset,  feature_extractor)

batch_size = 5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SpeechClassifier(nn.Module):
    def __init__(self, num_labels, encoder):
        super(SpeechClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_features, decoder_input_ids):
        outputs = self.encoder(input_features, decoder_input_ids=decoder_input_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits
num_labels = 2

model = SpeechClassifier(num_labels, encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()
# Define the training function NO VAL
def train(model, train_loader, optimizer, criterion, device, num_epochs):

  for epoch in range(num_epochs):

    model.train()

    for i, batch in enumerate(train_loader):

          input_features, decoder_input_ids, labels = batch

          input_features = input_features.squeeze()
          input_features = input_features.to(device)

          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)

          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)

          optimizer.zero_grad()

          logits = model(input_features, decoder_input_ids)

          loss = criterion(logits, labels)
          loss.backward()

          optimizer.step()

          if (i+1) % 8 == 0:
              print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), 'best_model.pt')
# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        for i, batch in enumerate(train_loader):
            input_features, decoder_input_ids, labels = batch
            input_features = input_features.squeeze()
            input_features = input_features.to(device)
            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)
            labels = labels.view(-1)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            optimizer.zero_grad()
            logits = model(input_features, decoder_input_ids)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0
        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')
        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, Best Accuracy: {best_accuracy:.4f}')
        print("========================================================================================")
def evaluate(model, data_loader,  device):
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
          input_features, decoder_input_ids, labels = batch
          input_features = input_features.squeeze()
          input_features = input_features.to(device)
          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)
          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)
          optimizer.zero_grad()
          logits = model(input_features, decoder_input_ids)
          loss = criterion(logits, labels)
          total_loss += loss.item()
          _, preds = torch.max(logits, 1)
          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds

import librosa
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

/tmp/ipykernel_1499/1211280835.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Batch 8/41, Train Loss: 0.8607
Epoch 1/5, Batch 16/41, Train Loss: 0.8422
Epoch 1/5, Batch 24/41, Train Loss: 0.5878
Epoch 1/5, Batch 32/41, Train Loss: 0.2673
Epoch 1/5, Batch 40/41, Train Loss: 0.2562
Epoch 1/5, Val Loss: 0.4632, Val Accuracy: 0.7528, Val F1: 0.7456, Best Accuracy: 0.7528
Epoch 2/5, Batch 8/41, Train Loss: 0.1017
Epoch 2/5, Batch 16/41, Train Loss: 0.0276
Epoch 2/5, Batch 24/41, Train Loss: 0.0145
Epoch 2/5, Batch 32/41, Train Loss: 0.0206
Epoch 2/5, Batch 40/41, Train Loss: 0.2250
Epoch 2/5, Val Loss: 0.1881, Val Accuracy: 0.9101, Val F1: 0.9075, Best Accuracy: 0.9101
Epoch 3/5, Batch 8/41, Train Loss: 0.0906
Epoch 3/5, Batch 16/41, Train Loss: 0.0427
Epoch 3/5, Batch 24/41, Train Loss: 0.0074
Epoch 3/5, Batch 32/41, Train Loss: 0.1029
Epoch 3/5, Batch 40/41, Train Loss: 0.0360
Epoch 3/5, Val Loss: 0.0562, Val Accuracy: 0.9888, Val F1: 0.9887, Best Accuracy: 0.9888
Epoch 4/5, Batch 8/41, Train Loss: 0.0543
Epoch 4/5, Batch 16/41, Train Loss: 0.0332
Epoch 

In [10]:
#VALIDATION
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, val_loader, device)

#VALIDATION
print("================")
print("Validatation\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

# TESTING ONLY
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, test_loader, device)

print("================")
print("Testing\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

Validatation


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        48

    accuracy                           1.00        89
   macro avg       1.00      1.00      1.00        89
weighted avg       1.00      1.00      1.00        89

1.0
Testing


              precision    recall  f1-score   support

           0       0.95      1.00      0.97        36
           1       1.00      0.95      0.97        38

    accuracy                           0.97        74
   macro avg       0.97      0.97      0.97        74
weighted avg       0.97      0.97      0.97        74

0.972972972972973


# ==================================================================================================================
# Large

In [11]:
data_path = "/workspace/cleft_palate_choja/WAV_PUBLIC_SAMPLES/NOISE"

train_catalog = "/workspace/cleft_palate_choja/train_noise.csv"
test_catalog = "/workspace/cleft_palate_choja/test_noise.csv"
model_checkpoint = "openai/whisper-large-v2"

train_metadata = pd.read_csv(train_catalog)
train_df, val_df = train_test_split(train_metadata, test_size = 0.3, random_state = 42)
train_files = train_df["WAV_filename"].tolist()
train_folder = train_df["WAV_folder"].tolist()
train_full_paths = [os.path.join(data_path,train_folder[i], train_files[i]) for i in range(0,len(train_files))]
train_labels = train_df["hypernasality"].tolist()
# val set
val_files = val_df["WAV_filename"].tolist()

val_folder = val_df["WAV_folder"].tolist()

val_full_paths = [os.path.join(data_path,val_folder[i], val_files[i]) for i in range(0,len(val_files))]

val_labels = val_df["hypernasality"].tolist()

test_metadata = pd.read_csv(test_catalog)
# add cols for wav data

# Replace ".mp3" with ".wav" in the "Filename" column
test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
test_metadata['WAV_folder'] = test_metadata['folder'] + "_WAV"

test_files = test_metadata["WAV_filename"].tolist()

test_folder = test_metadata["WAV_folder"].tolist()

test_full_paths = [os.path.join(data_path,test_folder[i], test_files[i]) for i in range(0,len(test_files))]

#test_full_paths

test_labels = test_metadata["hypernasality"].tolist()

train_audio_dataset = datasets.Dataset.from_dict({"audio": train_full_paths,
                                                  "labels":train_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
test_audio_dataset = datasets.Dataset.from_dict({"audio": test_full_paths,
                                                  "labels": test_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))
val_audio_dataset = datasets.Dataset.from_dict({"audio": val_full_paths,
                                                 "labels": val_labels }
                                             ).cast_column("audio", Audio(sampling_rate=16_000))
#model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

      inputs = self.text_processor(self.audio_data[index]["audio"]["array"],
                                   return_tensors="pt",
                                   sampling_rate=self.audio_data[index]["audio"]["sampling_rate"])
      input_features = inputs.input_features
      decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

      labels = np.array(self.audio_data[index]['labels'])

      return input_features, decoder_input_ids, torch.tensor(labels)
train_dataset = SpeechClassificationDataset(train_audio_dataset,  feature_extractor)
test_dataset = SpeechClassificationDataset(test_audio_dataset,  feature_extractor)
val_dataset = SpeechClassificationDataset(val_audio_dataset,  feature_extractor)

batch_size = 5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SpeechClassifier(nn.Module):
    def __init__(self, num_labels, encoder):
        super(SpeechClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_features, decoder_input_ids):
        outputs = self.encoder(input_features, decoder_input_ids=decoder_input_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits
num_labels = 2

model = SpeechClassifier(num_labels, encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()
# Define the training function NO VAL
def train(model, train_loader, optimizer, criterion, device, num_epochs):

  for epoch in range(num_epochs):

    model.train()

    for i, batch in enumerate(train_loader):

          input_features, decoder_input_ids, labels = batch

          input_features = input_features.squeeze()
          input_features = input_features.to(device)

          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)

          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)

          optimizer.zero_grad()

          logits = model(input_features, decoder_input_ids)

          loss = criterion(logits, labels)
          loss.backward()

          optimizer.step()

          if (i+1) % 8 == 0:
              print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), 'best_model.pt')
# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        for i, batch in enumerate(train_loader):
            input_features, decoder_input_ids, labels = batch
            input_features = input_features.squeeze()
            input_features = input_features.to(device)
            decoder_input_ids = decoder_input_ids.squeeze()
            decoder_input_ids = decoder_input_ids.to(device)
            labels = labels.view(-1)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            optimizer.zero_grad()
            logits = model(input_features, decoder_input_ids)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0
        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')
        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, Best Accuracy: {best_accuracy:.4f}')
        print("========================================================================================")
def evaluate(model, data_loader,  device):
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
          input_features, decoder_input_ids, labels = batch
          input_features = input_features.squeeze()
          input_features = input_features.to(device)
          decoder_input_ids = decoder_input_ids.squeeze()
          decoder_input_ids = decoder_input_ids.to(device)
          labels = labels.view(-1)
          labels = labels.type(torch.LongTensor)
          labels = labels.to(device)
          optimizer.zero_grad()
          logits = model(input_features, decoder_input_ids)
          loss = criterion(logits, labels)
          total_loss += loss.item()
          _, preds = torch.max(logits, 1)
          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds

import librosa
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

/tmp/ipykernel_1499/221531598.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  test_metadata['WAV_filename'] = test_metadata['File_Name'].str.replace('.mp3', '.wav')
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Batch 8/41, Train Loss: 0.7533
Epoch 1/5, Batch 16/41, Train Loss: 0.4578
Epoch 1/5, Batch 24/41, Train Loss: 0.5272
Epoch 1/5, Batch 32/41, Train Loss: 0.1500
Epoch 1/5, Batch 40/41, Train Loss: 2.1025
Epoch 1/5, Val Loss: 0.7166, Val Accuracy: 0.6966, Val F1: 0.6448, Best Accuracy: 0.6966
Epoch 2/5, Batch 8/41, Train Loss: 0.1206
Epoch 2/5, Batch 16/41, Train Loss: 0.0607
Epoch 2/5, Batch 24/41, Train Loss: 0.0247
Epoch 2/5, Batch 32/41, Train Loss: 0.0311
Epoch 2/5, Batch 40/41, Train Loss: 0.0240
Epoch 2/5, Val Loss: 0.1101, Val Accuracy: 0.9438, Val F1: 0.9438, Best Accuracy: 0.9438
Epoch 3/5, Batch 8/41, Train Loss: 0.0216
Epoch 3/5, Batch 16/41, Train Loss: 0.0250
Epoch 3/5, Batch 24/41, Train Loss: 0.0055
Epoch 3/5, Batch 32/41, Train Loss: 0.0047
Epoch 3/5, Batch 40/41, Train Loss: 0.0061
Epoch 3/5, Val Loss: 0.1294, Val Accuracy: 0.9551, Val F1: 0.9544, Best Accuracy: 0.9551
Epoch 4/5, Batch 8/41, Train Loss: 0.0440
Epoch 4/5, Batch 16/41, Train Loss: 0.0635
Epoch 

In [12]:
#VALIDATION
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, val_loader, device)

#VALIDATION
print("================")
print("Validatation\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

# TESTING ONLY
state_dict = torch.load('best_model.pt')

# Create a new instance of the model and load the state dictionary
num_labels = 2
model = SpeechClassifier(num_labels, encoder).to(device)
model.load_state_dict(state_dict)

_, _, _, all_labels, all_preds = evaluate(model, test_loader, device)

print("================")
print("Testing\n\n")
print(classification_report(all_labels, all_preds))
print(accuracy_score(all_labels, all_preds))

Validatation


              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.98      1.00      0.99        48

    accuracy                           0.99        89
   macro avg       0.99      0.99      0.99        89
weighted avg       0.99      0.99      0.99        89

0.9887640449438202
Testing


              precision    recall  f1-score   support

           0       0.90      1.00      0.95        36
           1       1.00      0.89      0.94        38

    accuracy                           0.95        74
   macro avg       0.95      0.95      0.95        74
weighted avg       0.95      0.95      0.95        74

0.9459459459459459
